In [2]:
import numpy as np
import csv

class reader(object):
  def __init__(self, data_file = "./data/wdbc.data"):
    self.value = []
    with open(data_file, "rb") as f:
      csv_reader = csv.reader(f, delimiter=",")
      for i, row in enumerate(csv_reader):
        self.value.append(row)
    # print self.value[0]
    # print self.value[0][0]
    # print self.value[0][1]

    self.raw_to_vector(self.value)
    self.split(num_validation_examples=50)
    
#     print self.id.shape
    print self.x.shape
    print self.y.shape
    
    self.num_examples = len(self.x_train)
    self.start_index = 0
    self.shuffle_indices = range(self.num_examples)

    self.num_examples_val = len(self.x_val) # = 50 (validation_data)
    self.start_index_val = 0
    self.shuffle_indices_val = range(self.num_examples_val)
    print self.num_examples_val

  def raw_to_vector(self, value):
    self.id = []
    self.x = []
    self.y = []

    for row in self.value:
        x = np.zeros(30)
        for i in range(30):
            x[i] = float(row[i+2])
        if row[1] == "B":
            y = 0
        else:
            y = 1
        self.x.append(x)
        self.y.append(y)
        id = int(row[0])
        self.id.append(id)
        
    self.x, self.y, self.id = np.array(self.x), np.array(self.y), np.array(self.id)

  def split(self, num_validation_examples):
    self.x_train = self.x[ num_validation_examples: ]
    self.x_val = self.x[ : num_validation_examples ]

    self.y_train = self.y[ num_validation_examples: ]
    self.y_val = self.y[ : num_validation_examples ]

    self.id_train = self.id[ num_validation_examples: ]
    self.id_val = self.id[ :num_validation_examples ]

  def next_batch(self, batch_size, split="train"):

    if split == "train":
      if self.start_index == 0:
        np.random.shuffle(self.shuffle_indices) # shuffle indices

      end_index = min([self.num_examples, self.start_index + batch_size])
      batch_indices = [ self.shuffle_indices[idx] for idx in range(self.start_index, end_index) ]

      batch_x = self.x_train[ batch_indices ]
      batch_y = self.y_train[ batch_indices ]
      batch_id = self.id_train[ batch_indices ] 

      if end_index == self.num_examples:
        self.start_index = 0
      else: self.start_index = end_index

      return batch_x, batch_y, batch_id 

    elif split == "val":
      if self.start_index_val == 0:
        np.random.shuffle(self.shuffle_indices_val) # shuffle indices

      end_index = min([self.num_examples_val, self.start_index_val + batch_size])
      batch_indices = [ self.shuffle_indices_val[idx] for idx in range(self.start_index_val, end_index) ]

      batch_x = self.x_val[ batch_indices ]
      batch_y = self.y_val[ batch_indices ]
      batch_id = self.id_val[ batch_indices ] 

      if end_index == self.num_examples_val:
        self.start_index_val = 0
      else: self.start_index_val = end_index

      return batch_x, batch_y, batch_id  

In [11]:
#################
### Logistic Regression with  Tensorflow
#################
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn import datasets, model_selection, metrics
import math

### define hyperparameters
n_classes = 2
n_features = 30

learning_rate = 0.1
max_iter = 10000

### define input data
data1 = reader()

# Split the data into training/testing sets
x_train, x_test, y_train, y_test = model_selection.train_test_split(
  data1.x, data1.y, test_size=0.2, random_state=0)
print x_train.shape
print x_test.shape
print y_train.shape
print y_test.shape

### define graph
x = tf.placeholder(tf.float32, shape=[None, n_features], name="X")
y_label = tf.placeholder(tf.int32, shape=[None], name="Y_label")

# one-hot encoding
y_onehot = tf.one_hot(indices=y_label, depth=n_classes)

# hidden 1
W_1 = tf.Variable(
      tf.truncated_normal([n_features, 50],
                          stddev=1.0),
      name='weights1')
B_1 = tf.Variable(
      tf.zeros([50]),
      name='biases1')

# hidden 2
W_2 = tf.Variable(
      tf.truncated_normal([50, 50],
                          stddev=1.0),
      name='weights2')
B_2 = tf.Variable(
      tf.zeros([50]),
      name='biases2')

# hidden 3
W_3 = tf.Variable(
      tf.truncated_normal([50, n_classes],
                          stddev=1.0),
      name='weights3')
B_3 = tf.Variable(
      tf.zeros([n_classes]),
      name='biases3')

# h1.shape = (batch, 50)
h1 = tf.matmul(x, W_1) + B_1 
h1 = tf.nn.relu(h1)

# h2.shape = (batch, 50)
h2 = tf.matmul(h1, W_2) + B_2
h2 = tf.nn.relu(h2)

# h3.shape = (batch, 2)
h3 = tf.matmul(h2, W_3) + B_3
h3 = tf.nn.softmax(h3, dim=-1)

# ####################################
# logits = h3
# scores = tf.nn.softmax( logits )

# pred = tf.cast( tf.argmax(logits, 1), tf.int32 )
# accuracy = tf.reduce_mean( tf.cast( tf.equal(pred, y_label), tf.float32 ))

# ### define loss
# loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
# train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

prediction = h3
loss = tf.reduce_mean((y_onehot - prediction) ** 2)

optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
#optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    
train_op = optimizer.minimize(loss)

pred_index = tf.cast(tf.argmax(prediction, dimension=1), tf.int32)
correct_prediction = tf.equal(pred_index, y_label)
acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


### Starting sessions
with tf.Session() as sess:
  ## initialize variables
  init = tf.global_variables_initializer()
  sess.run(init)
  
  ## training
  for i in range(max_iter):
    _, accuracy_val, pred_val, loss_val = sess.run(
      [train_op, acc, pred_index, loss], 
      feed_dict={x: x_train, y_label: y_train})

    if i % 100 == 0:
      print('=========== Epoch: %d ===========' % i)
      print('Loss', loss_val)
      print('Accuracy', accuracy_val)
      print('Y_prediction ', pred_val[:10])
      print('True', y_train[:10])

    if math.isnan(loss_val):
      print('LOSS is NAN!')
      break

(569, 30)
(569,)
50
(455, 30)
(114, 30)
(455,)
(114,)
=========== Epoch: 0 ===========
('Loss', 0.64175826)
('Accuracy', 0.35824177)
('Y_prediction ', array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32))
('True', array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))
=========== Epoch: 100 ===========
('Loss', 0.64175826)
('Accuracy', 0.35824177)
('Y_prediction ', array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32))
('True', array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))
=========== Epoch: 200 ===========
('Loss', 0.64175826)
('Accuracy', 0.35824177)
('Y_prediction ', array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32))
('True', array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))
=========== Epoch: 300 ===========
('Loss', 0.64175826)
('Accuracy', 0.35824177)
('Y_prediction ', array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32))
('True', array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))
=========== Epoch: 400 ===========
('Loss', 0.64175826)
('Accuracy', 0.35824177)
('Y_prediction ', array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32))
(

=========== Epoch: 4100 ===========
('Loss', 0.64175826)
('Accuracy', 0.35824177)
('Y_prediction ', array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32))
('True', array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))
=========== Epoch: 4200 ===========
('Loss', 0.64175826)
('Accuracy', 0.35824177)
('Y_prediction ', array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32))
('True', array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))
=========== Epoch: 4300 ===========
('Loss', 0.64175826)
('Accuracy', 0.35824177)
('Y_prediction ', array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32))
('True', array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))
=========== Epoch: 4400 ===========
('Loss', 0.64175826)
('Accuracy', 0.35824177)
('Y_prediction ', array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32))
('True', array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))
=========== Epoch: 4500 ===========
('Loss', 0.64175826)
('Accuracy', 0.35824177)
('Y_prediction ', array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32))
('True', array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))


=========== Epoch: 8300 ===========
('Loss', 0.64175826)
('Accuracy', 0.35824177)
('Y_prediction ', array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32))
('True', array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))
=========== Epoch: 8400 ===========
('Loss', 0.64175826)
('Accuracy', 0.35824177)
('Y_prediction ', array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32))
('True', array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))
=========== Epoch: 8500 ===========
('Loss', 0.64175826)
('Accuracy', 0.35824177)
('Y_prediction ', array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32))
('True', array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))
=========== Epoch: 8600 ===========
('Loss', 0.64175826)
('Accuracy', 0.35824177)
('Y_prediction ', array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32))
('True', array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))
=========== Epoch: 8700 ===========
('Loss', 0.64175826)
('Accuracy', 0.35824177)
('Y_prediction ', array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32))
('True', array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0]))
